In [1]:
import pandas as pd
df = pd.read_csv("http://bit.ly/kaggletrain")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [3]:
df.shape

(891, 12)

In [4]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
df.corr().Survived

PassengerId   -0.005007
Survived       1.000000
Pclass        -0.338481
Age           -0.077221
SibSp         -0.035322
Parch          0.081629
Fare           0.257307
Name: Survived, dtype: float64

In [6]:
#based on correlation we will only use Pclass, Sex, Survived and Embarked columns for our model
df = df.loc[df.Embarked.notna(), ['Survived', 'Pclass', 'Sex', 'Embarked']]
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In [7]:
df.shape

(889, 4)

In [8]:
X = df.drop('Survived', axis = 'columns')
X_train = X.loc[0:500, :]
X_train.shape

(500, 3)

In [9]:
df.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [10]:
y = df.Survived
y_train = y.loc[0:500]
y_train.head()
y_train.shape

(500,)

In [11]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
column_transform = make_column_transformer((OneHotEncoder(), ['Sex', 'Embarked']), remainder = 'passthrough')

In [12]:
column_transform.fit_transform(X_train)

array([[0., 1., 0., 0., 1., 3.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 3.],
       ...,
       [1., 0., 0., 0., 1., 1.],
       [0., 1., 0., 0., 1., 3.],
       [0., 1., 0., 0., 1., 3.]])

In [13]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [14]:
logreg = LogisticRegression()

In [15]:
pipeline = make_pipeline(column_transform, logreg)
cross_val_score(pipeline, X_train, y_train, cv = 10, scoring = 'accuracy').mean()

0.8

In [16]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['Sex', 'Embarked'])],
                                   verbose=False)),
                ('logisticregression',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                         

In [17]:
X_test = X.loc[501:, :]
X_test.shape

(389, 3)

In [18]:
y_test = y.loc[501:]
y_test.shape

(389,)

In [19]:
 y_pred = pipeline.predict(X_test)

In [20]:
from sklearn import metrics
metrics.confusion_matrix(y_test, y_pred)

array([[206,  35],
       [ 55,  93]])